# Colab SSH*

## SSH to Colab Instance

>**Warning :** This notebook is against the Policy of Colab. Use it on your own risk

***Testing Needed**


### Generate root password


In [ ]:
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

### Download ngrok

In [ ]:
!wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

!unzip -qq -n ngrok-stable-linux-amd64.zip

### Setup sshd

In [ ]:
!apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

### Set root password

In [ ]:
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

### Run sshd

In [ ]:
get_ipython().system_raw('/usr/sbin/sshd -D &')

### Ask token

In [ ]:
print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

### Create tunnel

In [ ]:
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

### Print root password

In [ ]:
print("Root password: {}".format(password))

### Get public address

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"